<a href="https://colab.research.google.com/github/dahwi/deepLearning/blob/main/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
import matplotlib.pyplot as plt

# Define transformations
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

# Download and load the training set
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
# Split the dataset into training and validation sets
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
train_subset, val_subset = random_split(trainset, [train_size, val_size])

trainloader = DataLoader(train_subset, batch_size=64, shuffle=True)
valloader = DataLoader(val_subset, batch_size=64, shuffle=False)

# Download and load the test set
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

In [36]:
class LeNet5(nn.Module):
    def __init__(self, dropout=False, batch_norm=False):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.conv3 = nn.Conv2d(16, 120, kernel_size=4)
        self.fc1 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, 10)

        self.dropout = nn.Dropout(0.5) if dropout else nn.Identity()
        self.batch_norm1 = nn.BatchNorm2d(6) if batch_norm else nn.Identity()
        self.batch_norm2 = nn.BatchNorm2d(16) if batch_norm else nn.Identity()

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = self.batch_norm1(x)
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = self.batch_norm2(x)
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv3(x))
        x = x.view(-1, 120)
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [38]:
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10, use_dropout_for_accuracy=True):
    best_val_accuracy = 0.0
    best_model_state = None
    train_accuracies = []
    val_accuracies = []

    for epoch in range(epochs):
        model.train()
        correct = 0
        total = 0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        # go over the training set and measure wo dropout
        if use_dropout_for_accuracy:
            model.eval()
            correct = 0
            total = 0
            with torch.no_grad():
                for images, labels in train_loader:
                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            model.train()
        train_accuracy = 100 * correct / total
        train_accuracies.append(train_accuracy)

        # Validation phase
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_accuracy = 100 * correct / total
        val_accuracies.append(val_accuracy)

        print(f"Epoch {epoch+1}/{epochs}, Train Accuracy: {train_accuracy}%, Validation Accuracy: {val_accuracy}%")

        # Save the best model
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_state = model.state_dict()

    # Load the best model state
    model.load_state_dict(best_model_state)
    return model, train_accuracies, val_accuracies

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy}%")
    return accuracy

In [34]:
criterion = nn.CrossEntropyLoss()
learning_rate = 1e-8

model = LeNet5()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_accuracies_no_reg = train_model(model, trainloader, criterion, optimizer)
test_accuracies_no_reg = evaluate_model(model, testloader)

Epoch 1/10, Train Accuracy: 78.00416666666666%
Epoch 2/10, Train Accuracy: 84.25%
Epoch 3/10, Train Accuracy: 85.68333333333334%
Epoch 4/10, Train Accuracy: 85.98958333333333%
Epoch 5/10, Train Accuracy: 86.12708333333333%
Epoch 6/10, Train Accuracy: 86.48125%
Epoch 7/10, Train Accuracy: 86.80208333333333%
Epoch 8/10, Train Accuracy: 86.84375%
Epoch 9/10, Train Accuracy: 87.09791666666666%
Epoch 10/10, Train Accuracy: 87.44583333333334%
Test Accuracy: 85.84%


In [ ]:
model_dropout = LeNet5(dropout=True)
optimizer = torch.optim.Adam(model_dropout.parameters(), lr=learning_rate)

train_accuracies_dropout = train_model(model_dropout, trainloader, criterion, optimizer, use_dropout_for_accuracy=False)
test_accuracies_dropout = evaluate_model(model_dropout, testloader)

In [ ]:
model_batchnorm = LeNet5(batch_norm=True)
optimizer = torch.optim.Adam(model_batchnorm.parameters(), lr=learning_rate)

train_accuracies_batchnorm = train_model(model_batchnorm, trainloader, criterion, optimizer)
test_accuracies_batchnorm = evaluate_model(model_batchnorm, testloader)

In [ ]:
model = LeNet5()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.01)

train_accuracies_weight_decay = train_model(model, trainloader, criterion, optimizer)
test_accuracies_weight_decay = evaluate_model(model, testloader)

In [ ]:
epochs = range(1, 20)

# Plotting convergence graphs
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(epochs, train_accuracies_no_reg, 'r', label='Train Accuracy')
plt.plot(epochs, [test_accuracies_no_reg]*len(epochs), 'r--', label='Test Accuracy')
plt.title('No Regularization')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(epochs, train_accuracies_dropout, 'g', label='Train Accuracy')
plt.plot(epochs, [test_accuracies_dropout]*len(epochs), 'g--', label='Test Accuracy')
plt.title('Dropout')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(2, 2, 3)
plt.plot(epochs, train_accuracies_batchnorm, 'b', label='Train Accuracy')
plt.plot(epochs, [test_accuracies_batchnorm]*len(epochs), 'b--', label='Test Accuracy')
plt.title('Batch Normalization')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(epochs, train_accuracies_weight_decay, 'y', label='Train Accuracy')
plt.plot(epochs, [test_accuracies_weight_decay]*len(epochs), 'y--', label='Test Accuracy')
plt.title('Weight Decay')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()